In [15]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [17]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [18]:
data = pd.read_csv('2003_mo_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value,Roraima - Consumo de Cimento (t)
0,2003-1,0.734916,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,6.128460e+06,332580.202821,12.101540,5.339136e+06,8.263084,0.294583,6.920
1,2003-2,0.734957,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,6.136622e+06,332914.755543,12.103485,5.342057e+06,8.257123,0.296505,7.476
2,2003-3,0.734998,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,6.144784e+06,333249.308265,12.105429,5.344978e+06,8.251161,0.298178,8.659
3,2003-4,0.735038,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,6.152947e+06,333583.860987,12.107373,5.347899e+06,8.245199,0.299598,7.259
4,2003-5,0.735079,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,6.161109e+06,333918.413709,12.109318,5.350821e+06,8.239238,0.300965,6.786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.581560,9.133
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.579849,10.622
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.577718,10.300
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575950,10.258


In [19]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
0,-0.346657,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.650047,-2.537523,-1.382546,-1.820180,-0.695651,-0.837394
1,-0.316945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.633241,-2.453668,-1.340990,-1.796532,-0.700839,-0.805077
2,-0.287234,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.616435,-2.369814,-1.299433,-1.772885,-0.706028,-0.776961
3,-0.257523,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.599629,-2.285960,-1.257877,-1.749238,-0.711217,-0.753079
4,-0.227812,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.582822,-2.202105,-1.216320,-1.725591,-0.716406,-0.730102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-3.166795,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.253468,-0.612057,-1.780820,1.251634,1.267469,1.568039
188,-3.318917,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.245588,-0.616438,-1.802446,1.242511,1.252097,1.600280
189,-3.471040,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.237708,-0.620818,-1.824072,1.233388,1.236726,1.635777
190,-3.623162,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.229828,-0.625199,-1.845698,1.224265,1.221354,1.674397


In [20]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      5.753
1      5.006
2      5.470
3      1.513
4      3.706
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Roraima - Consumo de Cimento (t), Length: 240, dtype: float64

In [21]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
0,-0.346657,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.650047,-2.537523,-1.382546,-1.820180,-0.695651,-0.837394
1,-0.316945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.633241,-2.453668,-1.340990,-1.796532,-0.700839,-0.805077
2,-0.287234,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.616435,-2.369814,-1.299433,-1.772885,-0.706028,-0.776961
3,-0.257523,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.599629,-2.285960,-1.257877,-1.749238,-0.711217,-0.753079
4,-0.227812,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.582822,-2.202105,-1.216320,-1.725591,-0.716406,-0.730102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.256163,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.210022,-0.560021,-0.995787,1.154680,1.464445,1.181073
158,-0.349321,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.218026,-0.570604,-1.003856,1.167190,1.447722,1.186391
159,-0.442479,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.226031,-0.581187,-1.011926,1.179699,1.430999,1.191228
160,-0.535637,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.234036,-0.591770,-1.019995,1.192209,1.414277,1.195247


In [22]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       5.753
1       5.006
2       5.470
3       1.513
4       3.706
        ...  
157     6.557
158     8.184
159     8.092
160     7.806
161    10.831
Name: Roraima - Consumo de Cimento (t), Length: 162, dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
162,-0.721952,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.250045,-0.612936,-1.036134,1.217228,1.380831,1.205280
163,-0.815110,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.258050,-0.623519,-1.044204,1.229737,1.364109,1.212037
164,-0.908268,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.266054,-0.634103,-1.052273,1.242247,1.347386,1.222566
165,-1.001426,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.274059,-0.644686,-1.060343,1.254756,1.330663,1.232837
166,-1.094584,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.282064,-0.655269,-1.068412,1.267266,1.313941,1.242845
167,-1.187741,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.290068,-0.665852,-1.076482,1.279775,1.297218,1.253922
168,-1.280899,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.298073,-0.676435,-1.084552,1.292285,1.280495,1.264859
169,-1.349319,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.298952,-0.668515,-1.129959,1.294219,1.288376,1.276145
170,-1.417739,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.299832,-0.660595,-1.175366,1.296153,1.296258,1.288500
171,-1.486159,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.300711,-0.652675,-1.220773,1.298087,1.304139,1.302831


In [24]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162     7.703
163     8.956
164     8.626
165     6.735
166     9.161
167     8.634
168     7.115
169     2.781
170     7.490
171     7.043
172     4.877
173     7.031
174     5.988
175     7.275
176     9.468
177     7.970
178     7.866
179     8.108
180     9.908
181    10.723
182    10.625
183    10.593
184    10.681
185     7.887
186     9.751
187     9.654
188     9.422
189    11.160
190    10.491
191    10.225
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [25]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [26]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1574598183, 2962728568, 231563765, 2911281399, 1290892090, 2513186850, 11096665, 3675812595, 182842898, 4129696714, 2277004759, 2462759476, 555022632, 2896217493, 2873501662, 2969039544, 3237986964, 2445977527, 679796467, 3868082559, 1282108530, 563545290, 2458019649, 3714108359, 3933127504]


Step: 0 ___________________________________________
val_loss: 1.6177871227264404
winner_seed: 1574598183


Step: 1 ___________________________________________
val_loss: 1.6074596643447876
winner_seed: 2962728568


Step: 2 ___________________________________________
val_loss: 1.6303025484085083


Step: 3 ___________________________________________
val_loss: 1.5620968341827393
winner_seed: 2911281399


Step: 4 ___________________________________________
val_loss: 1.6106880903244019


Step: 5 ___________________________________________
val_loss: 1.554144263267517
winner_seed: 2513186850


Step: 6 ___________________________________________
val_loss: 1.4366912841796875
winner_seed: 11096665


Step: 

In [29]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 20ms/step - loss: 31.5866 - val_loss: 14.6143
Epoch 2/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.5034 - val_loss: 4.9346
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.2078 - val_loss: 1.9215
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.5032 - val_loss: 1.9901
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.5543 - val_loss: 2.2210
Epoch 6/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.5659 - val_loss: 2.5448
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.7920 - val_loss: 2.0882
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.4738 - val_loss: 3.8160
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.8054 - val_loss: 1.9554
Epoch 10/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.0114 - val_loss: 2.2069
Epoch

Epoch 83/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1580 - val_loss: 1.6312
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2894 - val_loss: 1.7255
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0675 - val_loss: 1.6735
Epoch 86/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0744 - val_loss: 1.6984
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2880 - val_loss: 1.6469
Epoch 88/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1675 - val_loss: 1.9436
Epoch 89/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2738 - val_loss: 1.7866
Epoch 90/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1004 - val_loss: 1.7632
Epoch 91/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0406 - val_loss: 1.8751
Epoch 92/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1480 - val_loss: 1.9732


5/5 [==============================] - 0s 4ms/step - loss: 1.1265 - val_loss: 1.7258
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1499 - val_loss: 1.6768
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0284 - val_loss: 2.0696
Epoch 167/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0200 - val_loss: 1.6758
Epoch 168/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0920 - val_loss: 1.7994
Epoch 169/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0503 - val_loss: 1.7095
Epoch 170/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0706 - val_loss: 1.9147
Epoch 171/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0158 - val_loss: 2.1247
Epoch 172/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0804 - val_loss: 1.8170
Epoch 173/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0335 - val_loss: 1.6905
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 0.9897 - val_loss: 1.7808
Epoch 246/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0655 - val_loss: 1.9252
Epoch 247/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0500 - val_loss: 1.8662
Epoch 248/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0409 - val_loss: 1.7480
Epoch 249/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0308 - val_loss: 1.7704
Epoch 250/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0426 - val_loss: 1.7526
Epoch 251/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0706 - val_loss: 1.6703
Epoch 252/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0683 - val_loss: 1.6893
Epoch 253/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0578 - val_loss: 1.7869
Epoch 254/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.9700 - val_loss: 1.7765
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 1.0072 - val_loss: 1.7063
Epoch 327/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0455 - val_loss: 1.6829
Epoch 328/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0676 - val_loss: 1.7486
Epoch 329/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.9926 - val_loss: 1.7669
Epoch 330/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0399 - val_loss: 1.7118
Epoch 331/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.9534 - val_loss: 1.8009
Epoch 332/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0519 - val_loss: 1.8553
Epoch 333/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.9939 - val_loss: 1.8642
Epoch 334/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.0681 - val_loss: 1.8117
Epoch 335/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.9887 - val_loss: 1.7962
Epoch 

...

5/5 [==============================] - 0s 4ms/step - loss: 0.5247 - val_loss: 1.5042
Epoch 1935/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4048 - val_loss: 1.6540
Epoch 1936/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4165 - val_loss: 1.6200
Epoch 1937/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3951 - val_loss: 1.6174
Epoch 1938/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3351 - val_loss: 1.6960
Epoch 1939/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3676 - val_loss: 1.6286
Epoch 1940/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4290 - val_loss: 1.5275
Epoch 1941/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4544 - val_loss: 1.6948
Epoch 1942/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4029 - val_loss: 1.8416
Epoch 1943/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3987 - val_loss: 1.60

5/5 [==============================] - 0s 4ms/step - loss: 0.3542 - val_loss: 1.6446
Epoch 2015/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3852 - val_loss: 1.7011
Epoch 2016/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4920 - val_loss: 1.6834
Epoch 2017/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4731 - val_loss: 1.7614
Epoch 2018/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4204 - val_loss: 1.6399
Epoch 2019/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4374 - val_loss: 1.6474
Epoch 2020/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4734 - val_loss: 1.5933
Epoch 2021/10000
5/5 [==============================] - 0s 5ms/step - loss: 0.3953 - val_loss: 1.5325
Epoch 2022/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4549 - val_loss: 1.7736
Epoch 2023/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3855 - val_loss: 1.56

5/5 [==============================] - 0s 4ms/step - loss: 0.3743 - val_loss: 1.5585
Epoch 2095/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4115 - val_loss: 1.6279
Epoch 2096/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3965 - val_loss: 1.8277
Epoch 2097/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4114 - val_loss: 1.7834
Epoch 2098/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3705 - val_loss: 1.7340
Epoch 2099/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3816 - val_loss: 1.5807
Epoch 2100/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3858 - val_loss: 1.5353
Epoch 2101/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4041 - val_loss: 1.6980
Epoch 2102/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4347 - val_loss: 1.7078
Epoch 2103/10000
5/5 [==============================] - 0s 5ms/step - loss: 0.3185 - val_loss: 1.70

5/5 [==============================] - 0s 4ms/step - loss: 0.3731 - val_loss: 1.6503
Epoch 2175/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3435 - val_loss: 1.8074
Epoch 2176/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3594 - val_loss: 1.5542
Epoch 2177/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3468 - val_loss: 1.8437
Epoch 2178/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3104 - val_loss: 1.5931
Epoch 2179/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4301 - val_loss: 1.6820
Epoch 2180/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3627 - val_loss: 1.6220
Epoch 2181/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4158 - val_loss: 1.7639
Epoch 2182/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3610 - val_loss: 1.4922
Epoch 2183/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3923 - val_loss: 1.55

5/5 [==============================] - 0s 4ms/step - loss: 0.3482 - val_loss: 1.6747
Epoch 2255/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3253 - val_loss: 1.5992
Epoch 2256/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3468 - val_loss: 1.5470
Epoch 2257/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3157 - val_loss: 1.8358
Epoch 2258/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3952 - val_loss: 1.6617
Epoch 2259/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3692 - val_loss: 1.7025
Epoch 2260/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4070 - val_loss: 1.5936
Epoch 2261/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.4584 - val_loss: 1.5971
Epoch 2262/10000
5/5 [==============================] - 0s 4ms/step - loss: 0.3350 - val_loss: 1.6245
Epoch 2263/10000
5/5 [==============================] - 0s 5ms/step - loss: 0.3904 - val_loss: 1.48

In [30]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [31]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,8.19244,7.946839,7.790878,7.796916,7.676816,7.432786,4.36944,4.892592,5.477624,4.636312,2.536601,5.862472,1.784091,0.514437,2.742653,4.254889,3.511795,-0.280103,-1.65109,-1.592255,-2.600883,-3.348378,-4.16549,-4.502348,-5.750011,-9.053709,-9.990798,-9.163446,-7.970687,-7.587606
Target,7.703,8.956,8.626,6.735,9.161,8.634,7.115,2.781,7.49,7.043,4.877,7.031,5.988,7.275,9.468,7.97,7.866,8.108,9.908,10.723,10.625,10.593,10.681,7.887,9.751,9.654,9.422,11.16,10.491,10.225
Error,0.48944,1.009161,0.835123,1.061916,1.484185,1.201214,2.74556,2.111593,2.012375,2.406688,2.340399,1.168529,4.203909,6.760563,6.725348,3.715111,4.354206,8.388103,11.55909,12.315255,13.225883,13.941379,14.846489,12.389349,15.501012,18.70771,19.412798,20.323446,18.461687,17.812607


In [32]:
display(mae)
display(mape)

8.050337

0.8759014

In [33]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[74.61172]] - Target[86.152]| =  Error: [[11.540283]]; MAPE:[[0.13395259]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[-5.3326836]] - Target[107.092]| =  Error: [[112.42468]]; MAPE:[[1.0497953]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[-49.516254]] - Target[60.703]| =  Error: [[110.21925]]; MAPE:[[1.8157135]]


[array([[11.540283]], dtype=float32),
 array([[112.42468]], dtype=float32),
 array([[110.21925]], dtype=float32)]

78.06141

0.9998205